In [23]:
import pyxdf
import mne
import numpy as np
streams, header = pyxdf.load_xdf("sub-P005/ses-S001/eeg/sub-P005_ses-S001_task-Default_run-001_eeg.xdf") #Example Data from Lab Recoder

Stream 1: Calculated effective sampling rate 184.9737 Hz is different from specified rate 250.0000 Hz.


In [3]:
streams

[{'info': defaultdict(list,
              {'name': ['obci_eeg1'],
               'type': ['EEG'],
               'channel_count': ['8'],
               'nominal_srate': ['250'],
               'channel_format': ['float32'],
               'source_id': ['openbcigui'],
               'version': ['1.1000000000000001'],
               'created_at': ['50625.090058278001'],
               'uid': ['0c9963df-1bea-4888-9b52-bd045eb5b77f'],
               'session_id': ['default'],
               'hostname': ['MRPrayads-MacBook-Pro.local'],
               'v4address': [None],
               'v4data_port': ['16572'],
               'v4service_port': ['16572'],
               'v6address': [None],
               'v6data_port': ['0'],
               'v6service_port': ['0'],
               'desc': [None],
               'stream_id': 1,
               'effective_srate': 184.9736729492807}),
  'footer': {'info': defaultdict(list,
               {'first_timestamp': ['52747.81840181'],
                'l

In [4]:
raw_data = streams[0]["time_series"].T #From Steam variable this query is EEG data

In [5]:
raw_data.shape

(8, 48510)

In [6]:
channels = ['CH1','CH2','CH3','CH4','CH5','CH6','CH7','CH8'] #Set your target EEG channel name
info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= 250 #OpenBCI Frequency acquistion
)
# Create MNE rawarray
raw_mne = mne.io.RawArray(raw_data, info, verbose=False)

In [7]:
raw_mne

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [24]:
raw_mne_dropped = raw_mne.drop_channels(ch_names=['CH1','CH2','CH3','CH4'],on_missing='raise')

In [35]:
raw_mne_dropped

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,4 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [25]:
event_index = streams[1]["time_series"].T[0] #Get all event marker of experiment 
event_timestamp = streams[1]["time_stamps"].T #Timestamp when event marked

In [26]:
event_index

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4], dtype=int32)

In [77]:
event_timestamp

array([2249108.87993639, 2249118.87505466, 2249138.8932878 ,
       2249158.88566658, 2249178.87531063, 2249198.87553832,
       2249208.87830529, 2249228.88474543, 2249248.87139385,
       2249268.87054996, 2249288.87444326, 2249298.88224341,
       2249318.87085258, 2249338.88132632, 2249358.88318716])

In [86]:
events = np.column_stack((np.array(event_timestamp, dtype = int),
                        np.zeros(len(event_timestamp), dtype = int),
                        np.array(event_index, dtype = int)))

events_id = { # Set up your event name
     'break': 0,
     'forward': 1,
     'left': 2,
     'right': 3,
     'backward': 4,
     
}


mne_epochs = mne.Epochs(raw_mne_dropped, events, 
        tmin= -0.5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax= 20.0,    # final timestamp (10 means set epoch duration 10 second)
        event_id =events_id,
        reject= dict(eeg = 40e-6),
        flat= dict(eeg = 1e-6),
        preload = True,
        baseline = (-0.5, 1)
        
    )



Not setting metadata
15 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 15 events and 5126 original time points ...
15 bad epochs dropped


/var/folders/_h/6lrczjc52778lcxwz_y5nvdc0000gp/T/ipykernel_12870/1326620556.py:15: RuntimeWarning: All epochs were dropped!
You might need to alter reject/flat-criteria or drop bad channels to avoid this. You can use Epochs.plot_drop_log() to see which channels are responsible for the dropping of epochs.
  mne_epochs = mne.Epochs(raw_mne_dropped, events,


In [76]:
mne_epochs

Number of events,0
Events,backward: 0break: 0forward: 0left: 0right: 0
Time range,-0.500 – 20.000 s
Baseline,-0.500 – 0.000 s


In [73]:
dropped_log = mne_epochs.plot_drop_log

In [72]:
dropped_log

<bound method BaseEpochs.plot_drop_log of <Epochs |  0 events (all good), -0.5 – 20 s, baseline -0.5 – 2 s, ~8 kB, data not loaded,
 'break': 0
 'forward': 0
 'left': 0
 'right': 0
 'backward': 0>>

In [71]:
mne_epochs.plot(scalings = 200)


RuntimeError: epochs.plot() can't run because this Epochs-object is empty. You might want to check Epochs.drop_log or Epochs.plot_drop_log() to see why epochs were dropped.